In [2]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from collections import Counter
import imutils
import pprint
from matplotlib import pyplot as plt
from PIL import Image
import os, sys

# img, mask 불러오기

In [3]:
# 플라스크 서버 연동할 때 경로랑 손가락 구분하는거 다시 생각해보기
def img_mask_read(save_source, design_nail_crops):
    # 사용자 전체 손 사진
    img = cv2.imread(save_source)
    # 잘려진 손톱 사진
    mask = cv2.imread(design_nail_crops)
    return img, mask

# img 영역 찾기

In [4]:
def find_img_finger_1(img):
    dst = img[178:331, 247:354].copy()
    height, width, channel = dst.shape
    x = 247
    y = 178
    return dst, x, y, height, width

def find_img_finger_2(img):
    dst = img[226:381, 389:511].copy()
    height, width, channel = dst.shape
    x = 389
    y = 226
    return dst, x, y, height, width

def find_img_finger_3(img):
    dst = img[300:447, 542:669].copy()
    height, width, channel = dst.shape
    x = 542
    y = 300
    return dst, x, y, height, width

def find_img_finger_4(img):
    dst = img[386:503, 670:786].copy()
    height, width, channel = dst.shape
    x = 670
    y = 386
    return dst, x, y, height, width

# 피부색으로 손톱 검출하기

In [5]:
def prety_print_data(color_info):
    for x in color_info:
        print(pprint.pformat(x))
        print()
        

        
def plotColorBar(colorInformation):
    # Create a 500x100 black image
    color_bar = np.zeros((100, 500, 3), dtype="uint8")

    top_x = 0
    for x in colorInformation:
        bottom_x = top_x + (x["color_percentage"] * color_bar.shape[1])

        color = tuple(map(int, (x['color'])))

        cv2.rectangle(color_bar, (int(top_x), 0),
                      (int(bottom_x), color_bar.shape[0]), color, -1)
        top_x = bottom_x
    return color_bar


def extractDominantColor(image, number_of_colors=5, hasThresholding=False):

    # Quick Fix Increase cluster counter to neglect the black(Read Article)
    if hasThresholding == True:
        number_of_colors += 1

    # Taking Copy of the image
    img = image.copy()

    # Convert Image into RGB Colours Space
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Reshape Image
    img = img.reshape((img.shape[0]*img.shape[1]), 3)

    # Initiate KMeans Object
    estimator = KMeans(n_clusters=number_of_colors, random_state=0)

    # Fit the image
    estimator.fit(img)

    # Get Colour Information
    colorInformation = getColorInformation(
        estimator.labels_, estimator.cluster_centers_, hasThresholding)
    return colorInformation

def getColorInformation(estimator_labels, estimator_cluster, hasThresholding=False):

    # Variable to keep count of the occurance of each color predicted
    occurance_counter = None

    # Output list variable to return
    colorInformation = []

    # Check for Black
    hasBlack = False

    # If a mask has be applied, remove th black
    if hasThresholding == True:

        (occurance, cluster, black) = removeBlack(
            estimator_labels, estimator_cluster)
        occurance_counter = occurance
        estimator_cluster = cluster
        hasBlack = black

    else:
        occurance_counter = Counter(estimator_labels)

    # Get the total sum of all the predicted occurances
    totalOccurance = sum(occurance_counter.values())

    # Loop through all the predicted colors
    for x in occurance_counter.most_common(len(estimator_cluster)):

        index = (int(x[0]))

        # Quick fix for index out of bound when there is no threshold
        index = (index-1) if ((hasThresholding & hasBlack)
                              & (int(index) != 0)) else index

        # Get the color number into a list
        color = estimator_cluster[index].tolist()

        # Get the percentage of each color
        color_percentage = (x[1]/totalOccurance)

        # make the dictionay of the information
        colorInfo = {"cluster_index": index, "color": color,
                     "color_percentage": color_percentage}

        # Add the dictionary to the list
        colorInformation.append(colorInfo)

    return colorInformation

def removeBlack(estimator_labels, estimator_cluster):

    # Check for black
    hasBlack = False

    # Get the total number of occurance for each color
    occurance_counter = Counter(estimator_labels)

    # Quick lambda function to compare to lists
    def compare(x, y): return Counter(x) == Counter(y)

    # Loop through the most common occuring color
    for x in occurance_counter.most_common(len(estimator_cluster)):

        # Quick List comprehension to convert each of RBG Numbers to int
        color = [int(i) for i in estimator_cluster[x[0]].tolist()]

        # Check if the color is [0,0,0] that if it is black
        if compare(color, [0, 0, 0]) == True:
            # delete the occurance
            del occurance_counter[x[0]]
            # remove the cluster
            hasBlack = True
            estimator_cluster = np.delete(estimator_cluster, x[0], 0)
            break

    return (occurance_counter, estimator_cluster, hasBlack)

def extractSkin(image):
    # Taking a copy of the image
    img = image.copy()
    # Converting from BGR Colours Space to HSV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Defining HSV Threadholds
    lower_threshold = np.array([0, 48, 80], dtype=np.uint8)
    upper_threshold = np.array([20, 255, 255], dtype=np.uint8)

    # Single Channel mask,denoting presence of colours in the about threshold
    skinMask = cv2.inRange(img, lower_threshold, upper_threshold)

    # Cleaning up mask using Gaussian Filter
    skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)

    # Extracting skin from the threshold mask
    skin = cv2.bitwise_and(img, img, mask=skinMask)

    # Return the Skin image
    return cv2.cvtColor(skin, cv2.COLOR_HSV2BGR)

In [6]:
# 손톱에서 피부 색만 제외한 부분 검은색으로 하기 
def find_design_nail(mask):
    nailBasic = imutils.resize(mask, width=250)
    # 손톱에서 피부색만 추출
    skin = extractSkin(nailBasic)
    dominantColors2 = extractDominantColor(skin, hasThresholding=True)
    prety_print_data(dominantColors2)
    # 추출해낸 피부색을 담고 있는 막대기
    colour_bar = plotColorBar(dominantColors2)
    return skin, mask

# 검은색 손톱부분 흰색으로 반전시키기

In [7]:
# skin 색상을 제외한 손톱의 검은 영역을 흰색으로 반전시키기 
def black_2_white(skin):
    # skin색상 부분만 그레이로 바꾼뒤 검은색 손톱 부분 제외하고 흰색으로 처리하기
    nailGray2 = cv2.cvtColor(skin, cv2.COLOR_BGR2GRAY)
    th2, mask2 = cv2.threshold(nailGray2, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    # 흰색부분이 마스크가 되기 위해서 검은색 손톱 부분을 흰색으로 반전
    mask_b_inv = cv2.bitwise_not(mask2)
    return mask_b_inv, mask2

# 흰색인 손톱 부분을 원래 디자인 사진에 적용시키기

In [8]:
def mask_extract(mask, mask_b_inv):
    height, width, channel = mask.shape
    mask_b_inv = cv2.resize(mask_b_inv, (width, height))
    
    #마스크에 해당하는 이미지만 추출하기
    img_fg = cv2.bitwise_and(mask, mask, mask=mask_b_inv)
    return img_fg

# 사용자 손톱 각도에 맞춰 회전, resize 하기

In [9]:
def rotate_resize_1(img_fg, height, width):
    # 회전하기
    img_fg_copy = img_fg.copy()
    h, w = img_fg_copy.shape[:2]
    
    nail = cv2.getRotationMatrix2D((w//2 , h//2 ), -20, 1)
    img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(w, h))
    cv2.imshow('nail', img_fg_rotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # resize
    img_fg_resize = cv2.resize(img_fg_rotate, (width+20, height+10))
    return img_fg_resize

## 중지

In [10]:
def rotate_resize_2(img_fg, height, width):
    # 회전하기
    img_fg_copy = img_fg.copy()
    h, w = img_fg_copy.shape[:2]
    
    nail = cv2.getRotationMatrix2D((w//2 , h//2 ), -20, 1)
    img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(w, h))
    cv2.imshow('nail', img_fg_rotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # resize
    img_fg_resize = cv2.resize(img_fg_rotate, (width+5, height+10))
    return img_fg_resize

## 약지

In [11]:
def rotate_resize_3(img_fg, height, width):
    # 회전하기
    img_fg_copy = img_fg.copy()
    h, w = img_fg_copy.shape[:2]
    
    nail = cv2.getRotationMatrix2D((w//2 , h//2 ), -10, 1)
    img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(w, h))
    cv2.imshow('nail', img_fg_rotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # resize
    img_fg_resize = cv2.resize(img_fg_rotate, (width+20, height+25))
    return img_fg_resize

## 새끼

In [12]:
def rotate_resize_4(img_fg, height, width):
    # 회전하기
    img_fg_copy = img_fg.copy()
    h, w = img_fg_copy.shape[:2]
    
    nail = cv2.getRotationMatrix2D((w//2 , h//2 ), -25, 1)
    img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(w, h))
    cv2.imshow('nail', img_fg_rotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # resize
    img_fg_resize = cv2.resize(img_fg_rotate, (width-4, height+36))
    return img_fg_resize

# 배경 부분 투명처리하고 합치기

In [13]:
def transparent(img_fg_resize, x, y):
    color_coverted = cv2.cvtColor(img_fg_resize, cv2.COLOR_BGR2RGB)
    pillow_mask = Image.fromarray(color_coverted)
    pil_mask = pillow_mask.convert("RGBA")
    datas = pil_mask.getdata()
    
    newData = []
    for item in datas:
        if item[0] < 10 and item[1] < 10 and item[2] < 10:
            newData.append((item[0], item[1], item[2], 0))
        else:
            newData.append(item)
    
    pil_mask.putdata(newData)
    
    # 여기는 플라스크 서버 연동하고 경로 변경해야함
    pillow_img = Image.open("./img/19.png")
    pillow_img.paste(pil_mask, (x, y), pil_mask)
    return pillow_img    

In [14]:
def transparent2(img_fg_resize, x, y):
    color_coverted = cv2.cvtColor(img_fg_resize, cv2.COLOR_BGR2RGB)
    pillow_mask = Image.fromarray(color_coverted)
    pil_mask = pillow_mask.convert("RGBA")
    datas = pil_mask.getdata()
    
    newData = []
    for item in datas:
        if item[0] < 10 and item[1] < 10 and item[2] < 10:
            newData.append((item[0], item[1], item[2], 0))
        else:
            newData.append(item)
    
    pil_mask.putdata(newData)
    
    # 여기는 플라스크 서버 연동하고 경로 변경해야함
    pillow_img = Image.open("./19.jpg")
    pillow_img.paste(pil_mask, (x, y), pil_mask)
    return pillow_img    

# 테스트

In [15]:
# 합성할 이미지
img = cv2.imread("./img/19.png") # 사용자 손 사진
mask = cv2.imread("./img/3.jpg") # 욜로 돌아간 뒤 잘라진 사진 

In [16]:
mask.shape

(161, 122, 3)

In [821]:
dst, x, y, height, width = find_img_finger_2(img)
print(x, y, height, width)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

389 226 155 122


In [822]:
skin, mask = find_design_nail(mask)

cv2.imshow('skin', skin)
cv2.imshow('mask', mask)

cv2.waitKey(0)
cv2.destroyAllWindows()

{'cluster_index': 4,
 'color': [165.63683818045945, 97.09563758389211, 81.95469798657912],
 'color_percentage': 0.29975013881177126}

{'cluster_index': 2,
 'color': [144.2269639268952, 80.45383216916855, 69.35384173763347],
 'color_percentage': 0.2878956135480289}

{'cluster_index': 3,
 'color': [177.90949302483887, 123.85539299081117, 108.12470227968785],
 'color_percentage': 0.1641588006662965}

{'cluster_index': 1,
 'color': [116.11789562645224, 56.146207479400076, 47.651595182759394],
 'color_percentage': 0.1308717379233759}

{'cluster_index': 0,
 'color': [144.03687943262412, 98.19078014184282, 92.8070921985824],
 'color_percentage': 0.11732370905052748}



In [823]:
mask_b_inv, mask2 = black_2_white(skin)

cv2.imshow('mask_b_inv', mask_b_inv)
cv2.imshow('mask2', mask2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [824]:
mask2.shape

(329, 250)

In [825]:
mask_b_inv.shape

(329, 250)

In [826]:
img_fg = mask_extract(mask, mask_b_inv)

cv2.imshow('img_fg', img_fg)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [691]:
img_fg.shape

(164, 114, 3)

In [692]:
img_fg_resize=rotate_resize_4(img_fg, height, width)

cv2.imshow('img_fg_resize', img_fg_resize)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [693]:
img_fg_resize.shape

(191, 118, 3)

In [694]:
# 회전하기
img_fg_copy = img_fg.copy()
height , width = img_fg_copy.shape[:2]
print(height, width)

164 114


In [695]:
img_fg_copy.shape

(164, 114, 3)

In [700]:
img_fg_resize = rotate_resize_4(img_fg, height, width)

In [696]:
nail = cv2.getRotationMatrix2D((weight//2 , height//2 ), -20, 1)
img_fg_rotate = cv2.warpAffine(img_fg_copy, nail,(weight, height))
cv2.imshow('nail', img_fg_rotate)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [697]:
# resize
img_fg_resize = cv2.resize(img_fg_rotate, (117, 177))

In [701]:
img_fg_resize.shape

(200, 110, 3)

In [702]:
pillow_image = transparent(img_fg_resize, x, y)
pillow_image.show()

# 테스트

In [18]:
# 합성할 이미지
img = cv2.imread("./img/19.png") # 사용자 손 사진
mask1 = cv2.imread("./img/3.jpg") # 욜로 돌아간 뒤 잘라진 사진 
mask2 = cv2.imread("./img/3-2.jpg")
mask3 = cv2.imread("./img/3-3.jpg")
mask4 = cv2.imread("./img/3-4.jpg")

In [19]:
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("mask1", mask1)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("mask2", mask2)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("mask3", mask3)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("mask4", mask4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
dst1, x1, y1, height1, width1 = find_img_finger_1(img)
print("x1, y1, height1, width1 : ", x1, y1, height1, width1)

x1, y1, height1, width1 :  247 178 153 107


In [21]:
dst2, x2, y2, height2, width2 = find_img_finger_2(img)
print("x, y, height, width : ", x2, y2, height2, width2)

x, y, height, width :  389 226 155 122


In [22]:
dst3, x3, y3, height3, width3 = find_img_finger_3(img)
print("x, y, height, width : ", x3, y3, height3, width3)

x, y, height, width :  542 300 147 127


In [23]:
dst4, x4, y4, height4, width4 = find_img_finger_4(img)
print("x, y, height, width : ", x4, y4, height4, width4)

x, y, height, width :  670 386 117 116


In [25]:
skin1, mask1 = find_design_nail(mask1)
mask_b_inv1, mask1_1 = black_2_white(skin1)
print("mask_b_inv.shape : ", mask_b_inv1.shape)
print("mask2.shape : ", mask1_1.shape)

skin2, mask2 = find_design_nail(mask2)
mask_b_inv2, mask2_1 = black_2_white(skin2)
print("mask_b_inv.shape : ", mask_b_inv2.shape)
print("mask2.shape : ", mask2_1.shape)

skin3, mask3 = find_design_nail(mask3)
mask_b_inv3, mask3_1 = black_2_white(skin3)
print("mask_b_inv.shape : ", mask_b_inv3.shape)
print("mask2.shape : ", mask3_1.shape)

skin4, mask4 = find_design_nail(mask4)
mask_b_inv4, mask4_1 = black_2_white(skin4)
print("mask_b_inv.shape : ", mask_b_inv4.shape)
print("mask2.shape : ", mask4_1.shape)
        

{'cluster_index': 4,
 'color': [165.63683818045945, 97.09563758389208, 81.9546979865791],
 'color_percentage': 0.29975013881177126}

{'cluster_index': 2,
 'color': [144.2269639268952, 80.45383216916855, 69.35384173763347],
 'color_percentage': 0.2878956135480289}

{'cluster_index': 3,
 'color': [177.90949302483887, 123.85539299081117, 108.12470227968785],
 'color_percentage': 0.1641588006662965}

{'cluster_index': 1,
 'color': [116.11789562645222, 56.146207479400076, 47.651595182759394],
 'color_percentage': 0.1308717379233759}

{'cluster_index': 0,
 'color': [144.03687943262412, 98.19078014184282, 92.8070921985824],
 'color_percentage': 0.11732370905052748}

mask_b_inv.shape :  (329, 250)
mask2.shape :  (329, 250)
{'cluster_index': 4,
 'color': [146.3124124649865, 87.70658263305357, 78.51978291316425],
 'color_percentage': 0.3239273834357558}

{'cluster_index': 1,
 'color': [128.67596251552726, 71.681043242635, 63.49655639607015],
 'color_percentage': 0.24840728420815245}

{'cluster_i

In [31]:
cv2.imshow("img_fg1", img_fg1)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("img_fg2", img_fg2)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("img_fg3", img_fg3)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("img_fg4", img_fg4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:


cv2.imshow("mask4_1", mask4_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
img_fg1 = mask_extract(mask1, mask_b_inv1)
print("img_fg.shape : ", img_fg1.shape)
print("height, width : ", height1, width1)
cv2.imshow("img_fg1", img_fg1)
cv2.waitKey(0)
cv2.destroyAllWindows()

img_fg2 = mask_extract(mask2, mask_b_inv2)
print("img_fg.shape : ", img_fg2.shape)
print("height, width : ", height2, width2)
cv2.imshow("img_fg2", img_fg2)
cv2.waitKey(0)
cv2.destroyAllWindows()

img_fg3 = mask_extract(mask3, mask_b_inv3)
print("img_fg.shape : ", img_fg3.shape)
print("height, width : ", height3, width3)
cv2.imshow("img_fg3", img_fg3)
cv2.waitKey(0)
cv2.destroyAllWindows()

img_fg4 = mask_extract(mask4, mask_b_inv4)
print("img_fg.shape : ", img_fg4.shape)
print("height, width : ", height4, width4)
cv2.imshow("img_fg4", img_fg4)
cv2.waitKey(0)
cv2.destroyAllWindows()

img_fg.shape :  (161, 122, 3)
height, width :  153 107
img_fg.shape :  (164, 114, 3)
height, width :  155 122
img_fg.shape :  (153, 106, 3)
height, width :  147 127
img_fg.shape :  (129, 89, 3)
height, width :  117 116


In [32]:
img_fg_resize1 = rotate_resize_1(img_fg1, height1, width1)
print("img_fg_resize.shape : ", img_fg_resize1.shape)
cv2.imshow("img_fg1", img_fg1)
cv2.waitKey(0)
cv2.destroyAllWindows()

img_fg_resize2 = rotate_resize_2(img_fg2, height2, width2)
print("img_fg_resize.shape : ", img_fg_resize2.shape)

img_fg_resize3 = rotate_resize_3(img_fg3, height3, width3)
print("img_fg_resize.shape : ", img_fg_resize3.shape)

img_fg_resize4 = rotate_resize_4(img_fg4, height4, width4)
print("img_fg_resize.shape : ", img_fg_resize4.shape)

img_fg_resize.shape :  (163, 127, 3)
img_fg_resize.shape :  (165, 127, 3)
img_fg_resize.shape :  (172, 147, 3)
img_fg_resize.shape :  (153, 112, 3)


In [33]:
pillow_image = transparent(img_fg_resize1, x1, y1)
pillow_image = pillow_image.convert("RGB")
pillow_image.show()
# pillow_image.save("19.jpg")
pillow_image = transparent2(img_fg_resize2, x2, y2)
pillow_image = pillow_image.convert("RGB")
pillow_image.show()
# pillow_image.save("19.jpg")
pillow_image = transparent2(img_fg_resize3, x3, y3)
pillow_image = pillow_image.convert("RGB")
pillow_image.show()
# pillow_image.save("19.jpg")
pillow_image = transparent2(img_fg_resize4, x4, y4)
pillow_image = pillow_image.convert("RGB")
# pillow_image.save("19.jpg")
pillow_image.show()
#pillow_image.save(result_source)